In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# # Load MNIST data
# (X_train, y_train), (X_val, y_val) = mnist.load_data()

In [ ]:
#Load Dataset

train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

# Explore the data
train.head()

## Data Preprocessing
### 1. Separate Features and Labels
Split the training data into features (x) and labels (y).

In [ ]:
X_train = train.drop('label', axis=1).values
y_train = train['label'].values

### 3. Reshaping the data
For training neural networks like CNNs, reshape the 1D arrays into 2D images (28x28 pixels) with a single colour channel.

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

### 2. Normalization
Normalize pixel values to the range [0,1] by dividing by 255. Neural netwoks perform better with normalized data.

In [ ]:
X_train = X_train.astype('float32') / 255.0
test = test.astype('float32') / 255.0

### Visualize a few examples from the dataset

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(X_train[i].reshape(28, 28), cmap='gray')
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off')
plt.show()

### 4. One-Hot Encoding (Optional)
If you are using deep learning model with a categorical output layer, convert labels into one-hot encoded values.

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)

# Split the training data into training and validation sets (for model evaluation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### Building the model

In [ ]:
# Create the improved model using the Input layer
def create_improved_model():
    model = Sequential()

    # Use Input layer to specify input shape
    model.add(Input(shape=(28, 28, 1)))

    # Add Convolutional layers with BatchNormalization
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Add Dense layers with Dropout
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))  # Dropout to prevent overfitting
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(10, activation='softmax'))  # Output layer for 10 classes
    
    return model

# Compile the model
model = create_improved_model()
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)
datagen.fit(X_train)

# Add ReduceLROnPlateau to reduce the learning rate if validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=128),
                    epochs=20, 
                    validation_data=(X_val, y_val),
                    callbacks=[reduce_lr],
                    verbose=2)

Using CNNs

### Model Evaluation and Predictions

Once the model is trained, you can evaluate it on the training/validation data and make predictions for the test set.

In [ ]:
# Predict labels for test data
predictions = model.predict(test)

# Convert predictions to label indices
predictions = np.argmax(predictions, axis=1)

# Prepare the submission file
submission = pd.DataFrame({'ImageId': np.arange(1, len(predictions) + 1), 'Label': predictions})
submission.to_csv('submission3.csv', index=False)

print("Submission file created successfully!")